In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# read in orders data, single_items_crosswalk from Data Collection notebook, 
# and bundles_crosswalk (manually created csv from associated bundled product info)
orders = pd.read_csv('/Users/josh/Documents/Data Science/Apsey Farms/orders_export_1.csv')
single_items_crosswalk = pd.read_csv('/Users/josh/Documents/Data Science/Apsey Farms/single_items_crosswalk.csv')
bundles_crosswalk = pd.read_csv('/Users/josh/Documents/Data Science/Apsey Farms/bundle_crosswalk.csv')

In [ ]:
pd.options.display.max_columns = 100
orders.head()

In [ ]:
orders.info()

In [ ]:
single_items_crosswalk.head()

In [ ]:
bundles_crosswalk.head()

## Create a single product crosswalk dataframe
Ultimately, we want one product crosswalk that we'll use to cross-reference order data. So, let's get bundles_crosswalk into the same format as single_items_crosswalk and combine the two datasets.

In [ ]:
# restructure bundles_crosswalk dataframe
bundles_crosswalk.drop('Contents', axis=1, inplace=True)
bundles_crosswalk['product_type'] = 'Bundle'
bundles_crosswalk.rename(columns={'Bundle Name':'item_name', 'Beef':'quantity_beef_lb', 
                                 'Pork':'quantity_pork_lb', 'Chicken':'quantity_chicken_lb', 
                                 'Turkey':'quantity_turkey_lb', 'Total Weight':'total_quantity_lb',
                                'Enterprise':'enterprise'},inplace=True)

In [ ]:
bundles_crosswalk.head()

In [ ]:
# combine crosswalk dataframes
product_crosswalk = pd.concat([single_items_crosswalk,bundles_crosswalk])
product_crosswalk.reset_index(drop=True, inplace=True)
product_crosswalk['Lineitem name'] = product_crosswalk['item_name']

In [ ]:
product_crosswalk.head()

## Preliminary cleaning of orders data

Let's start by dropping columns we don't need.

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 75

In [ ]:
orders.isnull().sum()

In [ ]:
# find columns with >50% of values missing
cols_over_half_missing_values = list(orders.columns[orders.isnull().sum()/len(orders) > 0.5])
cols_over_half_missing_values

In [ ]:
# for now let's keep some of these columns we might use in our analysis and drop the rest 
# we'll also drop additional columns that won't be useful (e.g. 'Currency' only has one value of USD)
drop_cols = cols_over_half_missing_values
remove_from_drop_cols = ['Discount Code','Shipping Method','Shipping City','Shipping Zip',
                                                'Shipping Province','Notes','Tags','Shipping Province Name']
add_to_drop_cols = ['Currency','Billing Street','Billing Address1','Billing Country','Payment Reference','Vendor',
                    'Outstanding Balance','Source']

for col in remove_from_drop_cols:
    drop_cols.remove(col)
    
for col in add_to_drop_cols:
    drop_cols.append(col)

In [ ]:
drop_cols

In [ ]:
# drop columns from orders dataset
orders.drop(drop_cols,axis=1,inplace=True)

In [ ]:
orders.head()

Let's convert the 'Created at' column to datetime, then create a new column with just the month and year of each order.

In [ ]:
orders['Created at'] = pd.to_datetime(orders['Created at'], utc=True).dt.tz_convert('US/Eastern')

In [ ]:
orders['Created at'].dtype

In [ ]:
orders['order_month'] = orders['Created at'].dt.strftime('%Y-%m')
orders['order_month'] = pd.to_datetime(orders['order_month'])
orders['order_month'] = orders['order_month'].dt.date

Now let's explore some of our features to determine if there is additional cleaning we can do.

In [ ]:
# number of unique line items (products)
len(orders['Lineitem name'].unique())

In [ ]:
orders['Lineitem name'].value_counts()

Looks like we have some suspicious "products" e.g. UPS Shipping. Since "legit" products most likely contain certain words e.g. "beef", "pork", "chicken", let's filter those out of the 'Lineitem name' columns and investigate suspicious further.

In [ ]:
#dictionary version
suspicious_items_dict = {}
for item in orders['Lineitem name']:
    if not any(value in item.lower() for value in ('beef','pork','chicken','turkey','steak','bundle','box','bone','egg','steer','rib')):
        if item in suspicious_items_dict:
            suspicious_items_dict[item] += 1
        elif item not in suspicious_items_dict:
            suspicious_items_dict[item] = 1

# list version
suspicious_items_list = []
for item in orders['Lineitem name']:
    if not any(value in item.lower() for value in ('beef','pork','chicken','turkey','steak','bundle','box','bone','egg','steer','rib')):
        suspicious_items_list.append(item)

In [ ]:
suspicious_items_list

In [ ]:
# convert dict to df
suspicious_items_df = pd.DataFrame.from_dict(suspicious_items_dict, orient='index').reset_index()
suspicious_items_df = suspicious_items_df.rename(columns={'index':'Lineitem name', 0:'count'})
suspicious_items_df.sort_values('count', ascending=False)

Now that we've narrowed our list of suspicious products down, let's investigate them further to determine if they should be removed from our dataset for analysis.

In [ ]:
num_suspicious_lines = len(orders[orders['Lineitem name'].isin(suspicious_items_list)])
num_total_lines = len(orders)
print('Count of suspicious line items:', num_suspicious_lines)
print('Suspicious line items as a % of total line items:', round(num_suspicious_lines/num_total_lines*100,2))

In [ ]:
orders[orders['Lineitem name'].isin(suspicious_items_list)][['Lineitem name','Lineitem quantity','Total','Subtotal','Discount Amount',
                                                       'Lineitem price','Created at']]

In [ ]:
# total amount paid from products
orders[orders['Lineitem name'].isin(suspicious_items_list)]['Total'].value_counts()

In [ ]:
# Lineitem names for suspicious_items with total = $0
orders[(orders['Lineitem name'].isin(suspicious_items_list)) & (orders['Total']==0)]['Lineitem name']

Since the majority of the suspicious_items have a total amount paid of $0 and suspicious_items only make up <3\% of the total number of line items, let's drop these rows from our dataset. 

In [ ]:
suspicious_items_index = list(orders[orders['Lineitem name'].isin(suspicious_items_list)].index)
orders = orders.drop(suspicious_items_index)

In [ ]:
# check that rows were dropped
orders[orders['Lineitem name'].isin(suspicious_items_list)]

We know that Apsey Farms sometimes gives away products for free for promotions, gifts, etc. We won't consider these giveaways to be "true" sales/orders, so let's drop them from our dataset.

In [ ]:
len(orders[orders['Total']==0])

In [ ]:
# check for orders with $0 total
orders[orders['Total']==0].head(10)

In [ ]:
free_giveaways_index = list(orders[orders['Total']==0].index)
orders = orders.drop(free_giveaways_index)

In [ ]:
# check that rows were dropped
orders[orders['Total']==0]

In [ ]:
# number of line items (our true orders/sales) we're left with for analysis
len(orders)

## Modify product crosswalk to be a comprehensive single source of truth
#### Add items from orders data not already in crosswalk to crosswalk

In [ ]:
# get unique item names from orders, convert to df, and add to crosswalk
unique_lineitem_names = pd.DataFrame(orders['Lineitem name'].unique(), columns=['Lineitem name'])
product_crosswalk_full = pd.concat([product_crosswalk,unique_lineitem_names])

In [ ]:
product_crosswalk_full

In [ ]:
# check for duplicates
product_crosswalk_full.duplicated(['Lineitem name']).sum()

In [ ]:
# remove duplicates, keeping the first entry as it contains the associated feature values
product_crosswalk_full = product_crosswalk_full.drop_duplicates(['Lineitem name'])

In [ ]:
# confirm duplicates were dropped (370 - 72 = 298)
product_crosswalk_full.shape

#### Use Fuzzy Matching to impute values for missing items. 

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
# first let's separate the items with known values from those with missing values
known_items = product_crosswalk_full[product_crosswalk_full['item_name'].notna()]['item_name']
missing_items = product_crosswalk_full[product_crosswalk_full['item_name'].isna()]['Lineitem name']
print(known_items[:5])
print('\n')
print(missing_items[:5])
print('\n')
print('Total # known items:',len(known_items))
print('Total # missing items:',len(missing_items))

In [ ]:
# use fuzzywuzzy.process.extractOne() to find the top known item match for each missing item
fuzzy_top_choice = {}
for item in missing_items:
    fuzzy_top_choice[item] = process.extractOne(item, known_items)
fuzzy_top_choice

For the most part, it appears that choices with a score of 90 or greater (the second value in the choice tuple) are a good match with the missing item. However, it looks like fuzzywuzzy didn't do as great when the score is less than 90. So, we'll impute the values for missing items using choices with a score greater than or equal to 90, and use a different fuzzy function (fuzz.token_set_ratio) for cases where the score was less than 90 to pull out the top 3 matches so that we can manually choose the best one.

In [ ]:
# use score_cutoff parameter to filter for good matches using process.extractOne
fuzzy_top_choice = {}
for item in missing_items:
    fuzzy_top_choice[item] = process.extractOne(item, known_items, score_cutoff=89)
fuzzy_top_choice

In [ ]:
# we'll use these libraries to extract keys with top values from dictionaries
import heapq
from operator import itemgetter

In [ ]:
# iterate through fuzzy_top_choice dictionary and create separate dataframes for good and bad matches

fuzzy_below_90 = pd.DataFrame()
good_matches = {}

for key, value in fuzzy_top_choice.items():
    # if score from fuzzy_top_choice was <90, find top three scores using fuzzy token_set_ratio 
    # and add to fuzzy_below_90 df
    if value == None:
        ratios = {}
        for item in known_items:
            ratios[item] = fuzz.token_set_ratio(key, item)
        # get top 3 choices, returned as list of tuples where 1st element in tuple is the choice name and 2nd is fuzzy score
        top_3_items = heapq.nlargest(3, ratios.items(), key=itemgetter(1))
        # select only the choice name, not the score
        top_3_choices = [i[0] for i in top_3_items]
        top_3_choices_dict = {key: top_3_choices}
        top_3_choices_df = pd.DataFrame.from_dict(top_3_choices_dict,orient='index',columns=['choice_1','choice_2',
                                                                                             'choice_3'])
        top_3_choices_df = top_3_choices_df.reset_index().rename(columns={'index':'Lineitem name'})
        fuzzy_below_90 = pd.concat([fuzzy_below_90,top_3_choices_df])
    
    # if score from fuzzy_top_choice was >=90, add choice to good_matches_df
    else:
        good_matches[key] = value
        good_matches_df = pd.DataFrame.from_dict(good_matches, orient='index', columns=['choice','score','index'])

In [ ]:
print('Number of "good" matches:',len(good_matches))
print('Number of "bad" matches:',len(fuzzy_below_90))

In [ ]:
good_matches_df = good_matches_df.reset_index().drop(['score','index'],axis=1)
good_matches_df = good_matches_df.rename(columns={'level_0':'Lineitem name','choice':'item_name'})
good_matches_df

In [ ]:
fuzzy_below_90 = fuzzy_below_90.reset_index().drop('index',axis=1)
fuzzy_below_90

In [ ]:
# export fuzzy_below_90 to csv so that we can manually label the best match
fuzzy_below_90.to_csv('fuzzy_below_90.csv')

Let's impute values for good matches using the matching items in product_crosswalk_full, then we'll add the good matches back to our product crosswalk. Note that we'll need to drop the old, non-imputed items from product crosswalk; we'll do this at the end, after we've added values for good and bad matches to the product crosswalk.

In [ ]:
good_matches_impute = pd.merge(left=good_matches_df, right=product_crosswalk_full, how='left', on='item_name')
good_matches_impute = good_matches_impute.drop('Lineitem name_y',axis=1).rename(columns={'Lineitem name_x':'Lineitem name'})
good_matches_impute

In [ ]:
product_crosswalk_final = pd.concat([product_crosswalk_full,good_matches_impute])

In [ ]:
product_crosswalk_final

We manually labeled the best choice for our fuzzy_below_90 matches. Let's pull the data back in and add it to our product crosswalk.

In [ ]:
fuzzy_below_90_labeled = pd.read_csv('/Users/josh/Documents/Data Science/Apsey Farms/fuzzy_below_90_labeled.csv')

In [ ]:
fuzzy_below_90_labeled.drop(['Unnamed: 0','choice_1','choice_2','choice_3'],axis=1,inplace=True)
fuzzy_below_90_labeled.rename(columns={'final_choice':'item_name'},inplace=True)

In [ ]:
fuzzy_below_90_labeled.head()

Now we'll impute values for our fuzzy_below_90 matches using the matching items in product_crosswalk_full, then add those matches back to our product crosswalk. 

In [ ]:
fuzzy_matches_impute = pd.merge(left=fuzzy_below_90_labeled, right=product_crosswalk_full, how='left', on='item_name')
fuzzy_matches_impute = fuzzy_matches_impute.drop('Lineitem name_y',axis=1).rename(columns={'Lineitem name_x':'Lineitem name'})
fuzzy_matches_impute

In [ ]:
fuzzy_matches_impute['product_type'].isna().sum()

Looks like we have a number of items that did not match a previously defined product, so we'll have to fill in values for these. Notes:
 * for "bulk" items, the 'Lineitem quantity' field in the orders data indicates the weight in pounds, rather than quantity of items ordered. We'll leave the quantity columns blank for now, and impute those values when we merge the product crosswalk back to our orders data.
 * items labeled "eggs" will not have an associated weight, rather we measure quantity by the dozen. So, we'll also leave the quantity columns blank for those items.

In [ ]:
fuzzy_matches_impute.to_csv('fuzzy_matches_impute.csv')

In [ ]:
fuzzy_matches_impute_labeled = pd.read_csv('/Users/josh/Documents/Data Science/Apsey Farms/fuzzy_matches_impute_labeled.csv')

In [ ]:
fuzzy_matches_impute_labeled

Let's merge these items into our product crosswalk, then drop the old, non-imputed/duplicate items.

In [ ]:
product_crosswalk_final = pd.concat([product_crosswalk_final,fuzzy_matches_impute_labeled])

In [ ]:
product_crosswalk_final

In [ ]:
product_crosswalk_final.reset_index(inplace=True)

In [ ]:
product_crosswalk_final.drop('index',axis=1,inplace=True)

In [ ]:
product_crosswalk_final['item_name'].isnull().sum()

In [ ]:
null_items = list(product_crosswalk_final[product_crosswalk_final['item_name'].isnull()].index)

In [ ]:
product_crosswalk_final = product_crosswalk_final.drop(null_items)

In [ ]:
# check that null_items were dropped
product_crosswalk_final['item_name'].isnull().sum()

In [ ]:
# check that product crosswalk contains 298 items
len(product_crosswalk_final)

#### Use weights specified in item names to update item attributes
For example, 'Lineitem Name' = 'Ground Beef - 6 lbs' would previously have been matched with the standard ground beef item and assumed its standard quantity of 1 lb; however, 'quantity_beef_lb' for this item should instead be 6 (lbs).

In [ ]:
# find all Lineitem name's containing 'lb' or 'oz'
# only include product_type = 'Single item' since we've manually populated values for some bundles and don't want
# values for bulk items
single_products = product_crosswalk_final[product_crosswalk_final['product_type']=='Single item']
products_with_quantity = single_products[(single_products['Lineitem name'].str.contains('lb')) | (single_products['Lineitem name'].str.contains('oz'))]
products_with_quantity

In [ ]:
import re

In [ ]:
# extract quantity
# pattern = r"[.+]\s[.+]\s(?P<quantity>[.+])"
# pattern_2 = r"(\d*\.?\d+[+]?[\s]?[-]?[\s]?[\d*\.?\d+]?[\.\d+]?)"
pattern = r"(?P<quantity>\d*\.?\d+[+]?[\s]?[-]?[\s]?[\d*]?[\.]?[\d*]?)\s?(?P<measure>lbs?|lb?|oz?)"
quantity_extract = products_with_quantity['Lineitem name'].str.extract(pattern, flags=re.I)
quantity_extract

In [ ]:
# add extract to products_with_quantity dataframe by joining on the index
products_quant_extracted = pd.merge(left=products_with_quantity, right=quantity_extract, how='left', left_index=True, right_index=True)
products_quant_extracted

In [ ]:
# drop rows with missing quantity values
drop_rows = list(products_quant_extracted[products_quant_extracted['quantity'].isna()].index)
products_quant_extracted = products_quant_extracted.drop(drop_rows)

In [ ]:
# check that rows were dropped
print(products_quant_extracted['quantity'].isna().sum())
print(len(products_quant_extracted))

In [ ]:
# turn ranges of values in the quantity column into a single value by taking the average of the range min and max
# then create a new column with this value
def find_avg_quantity(value):
    quants = str(value).split('-')
    if len(quants) == 1:
        return value
    elif len(quants) == 2:
        return (float(quants[0])+float(quants[1]))/2
    
products_quant_extracted['quantity_avg'] = products_quant_extracted['quantity'].apply(find_avg_quantity)

In [ ]:
products_quant_extracted.head()

In [ ]:
products_quant_extracted['quantity_avg'].value_counts()

In [ ]:
# remove '+' from quantity_avg
# products_quant_extracted['quantity_avg'] = products_quant_extracted['quantity_avg'].str.replace('+','')

def remove_plus_sign(value):
    if '+' in str(value):
        return value.replace('+','')
    else:
        return value

products_quant_extracted['quantity_avg'] = products_quant_extracted['quantity_avg'].apply(remove_plus_sign)

In [ ]:
products_quant_extracted.head(8)

In [ ]:
products_quant_extracted['measure'].value_counts()

In [ ]:
# convert quantity_avg's in oz to lbs
products_quant_extracted['quantity_avg'] = products_quant_extracted['quantity_avg'].astype('float')
products_quant_extracted['quantity_avg_lb'] = np.where(products_quant_extracted['measure']=='oz', 
                                         products_quant_extracted['quantity_avg']/16, products_quant_extracted['quantity_avg'])

In [ ]:
products_quant_extracted.head()

In [ ]:
# use quantity_avg to update quantity values
# excluding Turkey since we updated those manually previously
products_quant_extracted['quantity_beef_lb'] = np.where(products_quant_extracted['enterprise']=='Beef',
                                                     products_quant_extracted['quantity_avg_lb'],0)
products_quant_extracted['quantity_pork_lb'] = np.where(products_quant_extracted['enterprise']=='Pork',
                                                     products_quant_extracted['quantity_avg_lb'],0)
products_quant_extracted['quantity_chicken_lb'] = np.where(products_quant_extracted['enterprise']=='Chicken',
                                                     products_quant_extracted['quantity_avg_lb'],0)

In [ ]:
products_quant_extracted.head()

In [ ]:
# reset total_quantity_lb
products_quant_extracted['total_quantity_lb'] = products_quant_extracted['quantity_beef_lb']+products_quant_extracted['quantity_pork_lb']+products_quant_extracted['quantity_chicken_lb']+products_quant_extracted['quantity_turkey_lb']
products_quant_extracted.head()

In [ ]:
# drop unecessary columns
products_quant_extracted = products_quant_extracted.drop(['quantity','measure','quantity_avg','quantity_avg_lb'],
                                                        axis=1)
products_quant_extracted.head()

In [ ]:
products_quant_extracted.shape

Now that we've updated the quantity attributes for items that contained a quantity in their name, let's merge these items back into our product crosswalk.

In [ ]:
product_crosswalk_final = pd.concat([product_crosswalk_final,products_quant_extracted])

In [ ]:
product_crosswalk_final.duplicated(['Lineitem name']).sum()

In [ ]:
# we want to keep the last duplicate row, since products_quant_extracted was added to the end of the 
# product_crosswalk_final df
product_crosswalk_final.drop_duplicates(['Lineitem name'],keep='last',inplace=True)

In [ ]:
product_crosswalk_final.reset_index(inplace=True)

In [ ]:
product_crosswalk_final.drop('index',axis=1,inplace=True)

In [ ]:
product_crosswalk_final.head()

In [ ]:
product_crosswalk_final.shape

In [ ]:
product_crosswalk_final['product_type'].value_counts()

In [ ]:
product_crosswalk_final['enterprise'].value_counts()

In [ ]:
product_crosswalk_final.to_csv('product_crosswalk_final.csv')

## Add features to Orders data that we'll use in our analysis

In [ ]:
orders_clean = orders.copy()

In [ ]:
# merge orders with product crosswalk
orders_clean = pd.merge(left=orders_clean, right=product_crosswalk_final, how='left', on='Lineitem name')

In [ ]:
orders_clean.head()

In [ ]:
orders_clean[orders_clean['total_quantity_lb'].isna()]

In [ ]:
# create new columns with the total item weight and weight per enterprise
# note: doesn't apply to bulk items and eggs
orders_clean['total_item_weight'] = orders_clean['Lineitem quantity'] * orders_clean['total_quantity_lb']
orders_clean['item_weight_beef'] = orders_clean['Lineitem quantity'] * orders_clean['quantity_beef_lb']
orders_clean['item_weight_pork'] = orders_clean['Lineitem quantity'] * orders_clean['quantity_pork_lb']
orders_clean['item_weight_chicken'] = orders_clean['Lineitem quantity'] * orders_clean['quantity_chicken_lb']
orders_clean['item_weight_turkey'] = orders_clean['Lineitem quantity'] * orders_clean['quantity_turkey_lb']

In [ ]:
orders_clean.head(10)

In [ ]:
orders_clean.to_csv('orders_clean.csv')